In [366]:
import pandas as pd
import numpy as np
import json
from hatchet import *
import pprint
import sys

In [304]:
def read_file(jsonfile):
  f = open(jsonfile, 'r')
  path = json.loads(f.read())['path']
  return path


In [68]:
def create_gf(paths, file_format):
  ret = []
  for i in range(0, len(paths)):
    gf = GraphFrame()
    if file_format == 'hpctoolkit':
      gf.from_hpctoolkit(paths[i])
    elif file_format == 'caliper':
      gf.from_caliper(paths[i])
    ret.append(gf)
  return ret


In [85]:
def getFileNames(gf):
    ret = []
    file_df = gf.dataframe.groupby('file')
    for key in file_df:
        ret.append(key[0])
    return ret

In [11]:
 def run(self, gfs):
    ret = {}
    graphID = 0

In [181]:
def aggregate(df):
    print df.values, type(df.keys)
    print type(df)

In [344]:
def metrics(df):
    metrics = {}
    print df[['name']], len(df[['name']])
    for key in list(df):
        metrics[key] = []
    for key in list(df):
        for procs in range(0,len(df[['name']])):
            metrics[key].append(df[key][0])
    return metrics

In [437]:
def node_lookup(gf, node):
    df = gf.dataframe
    return df.loc[df['node'] == node][['name','CPUTIME (usec) (I)']]

In [70]:
url_path = "/Users/kesavan1/Suraj/llnl/CallFlow/calc-pi.json"
data_paths = read_file(url_path)

In [424]:
pp = pprint.PrettyPrinter(indent=4)

In [439]:
for gf in gfs:
    root = gf.graph.roots[0]
    node_gen = gf.graph.roots[0].traverse()
    while root != None:
        root = next(node_gen)
        print root, node_lookup(gf, root)


<hatchet.node.Node instance at 0x10e06fb90>                                                             name  \
callpath                                    rank                   
<hatchet.node.Node instance at 0x10e06fb90> 0     <program root>   
                                            1     <program root>   
                                            2     <program root>   

                                                  CPUTIME (usec) (I)  
callpath                                    rank                      
<hatchet.node.Node instance at 0x10e06fb90> 0               999238.0  
                                            1              1000306.0  
                                            2               999308.0  
<hatchet.node.Node instance at 0x10e06fd88>                                                   name  CPUTIME (usec) (I)
callpath                                    rank                          
<hatchet.node.Node instance at 0x10e06fd88> 0     main            

StopIteration: 

In [384]:
for gf in gfs:
    name_df = gf.dataframe.groupby(['module','name'])
    nodes_map = {}
    max_inclusive_time = 0
    max_exclusive_time = 0
    num_pes = 3
    
    for key, item in name_df:
        max_inclusive_time = max(max_inclusive_time, item[['CPUTIME (usec) (I)']].max()[0])

    print max_inclusive_time
    
    print len(name_df.groups)      
    name_df.filter(lambda x: sys.stdout.write(str(x[['CPUTIME (usec) (I)']].max()[0])))
    name_df.filter(lambda x: x[['CPUTIME (usec) (I)']].max()[0] > 0.1*max_inclusive_time)
                   #x[['CPUTIME (usec) (E)']].sum()[0]/num_pes > 0.1*max_inclusive_time)    
    print len(name_df.groups)
    

1000306.0
24
1000306.01000306.00.00.017989.01000306.01000306.00.01000306.00.00.01000306.00.01000306.00.01000306.024


In [356]:
for gf in gfs:
    # Structure creation
    for key, item in name_df:
        node_key = key[0]
        nodes_map[node_key] = {}
        nodes_map[node_key]["fns"] = []
        print item[['CPUTIME (usec) (E)']].sum()

    for key, item in name_df:
        node_key = key[0]
        metrics_pes = []
        for rank_keys, rank_items in item.groupby(['rank']):
            metrics_pes.append(metrics(rank_items))

        nodes_map[node_key]["fns"].append({
            "fn_name": key[1],
            "metrics": metrics_pes
        })
    #print len(nodes_map[node_key]["fns"])

    nodes = []
    for key, item in nodes_map.iteritems():
        nodes.append({
            'module': key,
            'props': item
        })


CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    17989.0
dtype: float64
CPUTIME (usec) (E)    2980863.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
CPUTIME (usec) (E)    0.0
dtype: float64
                                                            name
callpath                                    rank                
<hatchet.node.Node instance at 0x10e06fb90> 0     <program root> 1
                                                            name
callpath                                    rank                
<hatche

<hatchet.node.Node instance at 0x10e061ea8> 0     PMPI_Init 1
                                                       name
callpath                                    rank           
<hatchet.node.Node instance at 0x10e061ea8> 1     PMPI_Init 1
                                                       name
callpath                                    rank           
<hatchet.node.Node instance at 0x10e061ea8> 2     PMPI_Init 1
                                                            name
callpath                                    rank                
<hatchet.node.Node instance at 0x10e624ef0> 0     psm_dofinalize 1
                                                            name
callpath                                    rank                
<hatchet.node.Node instance at 0x10e624ef0> 1     psm_dofinalize 1
                                                            name
callpath                                    rank                
<hatchet.node.Node instance at 0x10e624ef0> 2     

In [86]:
getFileNames(gfs[0])

['./src/cpi.c', '/tmp/dpkg-mkdeb.gouoc49UG7/src/mvapich/src/build/../src/mpi/init/finalize.c', '/tmp/dpkg-mkdeb.gouoc49UG7/src/mvapich/src/build/../src/mpi/init/init.c', '/tmp/dpkg-mkdeb.gouoc49UG7/src/mvapich/src/build/../src/mpi/init/initthread.c', '/tmp/dpkg-mkdeb.gouoc49UG7/src/mvapich/src/build/../src/mpid/ch3/channels/psm/src/mpidi_calls.c', '/tmp/dpkg-mkdeb.gouoc49UG7/src/mvapich/src/build/../src/mpid/ch3/channels/psm/src/psm_entry.c', '/tmp/dpkg-mkdeb.gouoc49UG7/src/mvapich/src/build/../src/mpid/ch3/channels/psm/src/psm_exit.c', '/tmp/dpkg-mkdeb.gouoc49UG7/src/mvapich/src/build/../src/mpid/ch3/src/mpid_finalize.c', '/tmp/dpkg-mkdeb.gouoc49UG7/src/mvapich/src/build/../src/mpid/ch3/src/mpid_init.c', '<unknown file>', 'interp.c']
